# Making stacked histograms
- processes an `events[year][ch][sample]` object using `make_events_dict()`
- uses `plot_hists()` to make stacked histograms

In [1]:
import glob
import os
import json
import pickle
import yaml
import math

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import pyarrow.parquet as pq
from sklearn.metrics import auc, roc_curve
from scipy.special import softmax

import hist as hist2
import matplotlib.pyplot as plt
import mplhep as hep

plt.style.use(hep.style.CMS)

import sys
sys.path.append("../python/")

import utils

plt.rcParams.update({"font.size": 20})

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
# get lumi
with open("../fileset/luminosity.json") as f:
    luminosity = json.load(f)
    
luminosity

{'ele': {'Run2': 137640.0,
  '2016APV': 19492.72,
  '2016': 16809.96,
  '2017': 41476.02,
  '2018': 59816.23},
 'mu': {'Run2': 137640.0,
  '2016APV': 19436.16,
  '2016': 16810.81,
  '2017': 41475.26,
  '2018': 59781.96},
 'lep': {'Run2': 137640.0,
  '2016APV': 19436.16,
  '2016': 16810.81,
  '2017': 41475.26,
  '2018': 59781.96},
 'had': {'Run2': 137640.0,
  '2016APV': 19436.16,
  '2016': 16810.81,
  '2017': 41475.26,
  '2018': 59781.96}}

In [4]:
def get_lumi(years, channels):
    lum_ = 0
    for year in years:
        lum = 0
        for ch in channels:
            lum += luminosity[ch][year] / 1000.0

        lum_ += lum / len(channels)    
    return lum_

In [87]:
 # define your regions here
presel = {
        "mu": {
            "lepmiso": "(lep_pt<55) | ( (lep_pt>=55) & (lep_misolation<0.8) )",   # needed for the fakes            

#             "jetvetomap": "jetvetomap==1",
#             "msoftdrop": "fj_mass>30",
            "msoftdrop": "fj_mass>40",
            "THWW": "THWW>0.75",
            
#             "tagger>0.50": "THWW<0.5",
#             "FakeValidation": "fj_mass>30 & fj_mass<40",
#             "msoftdrop": "fj_mass<40",
#             "met": "met_pt>20",

#             "bveto": "n_bjets_T>0",
        },
        "ele": {

#             "jetvetomap": "jetvetomap==1",            
#             "msoftdrop": "fj_mass>30",
            "msoftdrop": "fj_mass>40",
            "THWW": "THWW>0.75",

#             "tagger>0.50": "THWW<0.5",
#             "FakeValidation": "fj_mass>30 & fj_mass<40",
#             "met": "met_pt>20",
#             "lepmiso": "(lep_pt<120) | ( (lep_pt>120) & (lep_misolation<0.02))",
#             "bveto": "n_bjets_T>0",
        },
}

channels = [
    "ele",
    "mu",
]
samples = [
    "ggF", 
    "VBF",  
    "WH",
    "ZH",
    "ttH",
    "QCD",
    "WJetsLNu",
    "TTbar",
    "SingleTop",
    "Diboson",
    "EWKvjets",
    "DYJets",
    "WZQQ",

    "Data",
#     "Fake",
]

years = [
    "2016",
    "2016APV",
    "2017",
    "2018",
]

In [88]:
events_dict = {}

In [89]:
samples_dir = {
    "without": {
        "2016":    "../eos/Oct10_hww_2016",
        "2016APV": "../eos/Oct10_hww_2016APV",
        "2017":    "../eos/Oct10_hww_2017",
        "2018":    "../eos/Oct10_hww_2018",
    },
    "with": {
        "2016":    "../eos/Dec20_hww_2016",
        "2016APV": "../eos/Dec20_hww_2016APV",
        "2017":    "../eos/Dec20_hww_2017",
        "2018":    "../eos/Dec20_hww_2018",                
    }
}

THWW_path = "../../weaver-core-dev/experiments_finetuning/v35_30/model.onnx"

from make_stacked_hists import make_events_dict

for mode in ["with", "without"]:
    events_dict[mode] = make_events_dict(years, channels, samples_dir[mode], samples, presel, THWW_path, fake_SF={"ele": 0.75, "mu": 1})

INFO:root:Finding VBFHToWWToAny_M-125_TuneCP5_withDipoleRecoil samples and should combine them under VBF
INFO:root:Applying msoftdrop selection on 798 events
INFO:root:Applying THWW selection on 689 events
INFO:root:Will fill the VBF dataframe with the remaining 253 events
INFO:root:tot event weight 2.2697910741257665 

INFO:root:Finding fake_2016_ele_EWK_SF_Down.parquet samples and should combine them under EWKvjets
INFO:root:No parquet file for fake_2016_ele_EWK_SF_Down.parquet
INFO:root:Finding WJetsToLNu_HT-100To200 samples and should combine them under WJetsLNu
INFO:root:Applying msoftdrop selection on 19 events
INFO:root:Applying THWW selection on 12 events
INFO:root:Will fill the WJetsLNu dataframe with the remaining 0 events
INFO:root:tot event weight 0.0 

INFO:root:Finding SingleMuon_Run2016H samples and should combine them under Data
INFO:root:Finding SingleMuon_Run2016F samples and should combine them under Data
INFO:root:Finding EWKWminus_WToLNu samples and should combine 

INFO:root:Applying THWW selection on 4616 events
INFO:root:Will fill the SingleTop dataframe with the remaining 20 events
INFO:root:tot event weight 0.08818600841034617 

INFO:root:Finding DYJetsToLL_LHEFilterPtZ-50To100 samples and should combine them under DYJets
INFO:root:Applying msoftdrop selection on 1585 events
INFO:root:Applying THWW selection on 1539 events
INFO:root:Will fill the DYJets dataframe with the remaining 26 events
INFO:root:tot event weight 1.8132959916919125 

INFO:root:Finding QCD_Pt_1800to2400 samples and should combine them under QCD
INFO:root:Applying msoftdrop selection on 408 events
INFO:root:Applying THWW selection on 350 events
INFO:root:Will fill the QCD dataframe with the remaining 1 events
INFO:root:tot event weight 0.0003166747259336538 

INFO:root:Finding fake_2016_mu_EWK_SF_Down.parquet samples and should combine them under EWKvjets
INFO:root:No parquet file for fake_2016_mu_EWK_SF_Down.parquet
INFO:root:Finding WW samples and should combine them und

INFO:root:Finding WJetsToQQ_HT-400to600 samples and should combine them under WZQQ
INFO:root:Applying msoftdrop selection on 35 events
INFO:root:Applying THWW selection on 28 events
INFO:root:Will fill the WZQQ dataframe with the remaining 1 events
INFO:root:tot event weight 1.508073155159401 

INFO:root:Finding WJetsToLNu_HT-400To600 samples and should combine them under WJetsLNu
INFO:root:Applying msoftdrop selection on 11918 events
INFO:root:Applying THWW selection on 10463 events
INFO:root:Will fill the WJetsLNu dataframe with the remaining 251 events
INFO:root:tot event weight 135.11421713067188 

INFO:root:Finding QCD_Pt_470to600 samples and should combine them under QCD
INFO:root:Applying msoftdrop selection on 2057 events
INFO:root:Applying THWW selection on 1787 events
INFO:root:Will fill the QCD dataframe with the remaining 12 events
INFO:root:tot event weight 1.8715461699585343 

INFO:root:Finding HZJ_HToWW_M-125 samples and should combine them under ZH
INFO:root:Applying ms

INFO:root:Finding DYJetsToLL_LHEFilterPtZ-650ToInf samples and should combine them under DYJets
INFO:root:Applying lepmiso selection on 58406 events
INFO:root:Applying msoftdrop selection on 58406 events
INFO:root:Applying THWW selection on 44943 events
INFO:root:Will fill the DYJets dataframe with the remaining 442 events
INFO:root:tot event weight 0.15955145071807525 

INFO:root:Finding SingleElectron_Run2016F samples and should combine them under Data
INFO:root:Finding WJetsToQQ_HT-200to400 samples and should combine them under WZQQ
INFO:root:Finding ST_tW_antitop_5f_inclusiveDecays samples and should combine them under SingleTop
INFO:root:Applying lepmiso selection on 1976 events
INFO:root:Applying msoftdrop selection on 1976 events
INFO:root:Applying THWW selection on 1720 events
INFO:root:Will fill the SingleTop dataframe with the remaining 39 events
INFO:root:tot event weight 7.20566286925947 

INFO:root:Finding ZJetsToQQ_HT-200to400 samples and should combine them under WZQQ
IN

INFO:root:Applying THWW selection on 481 events
INFO:root:Will fill the WZQQ dataframe with the remaining 8 events
INFO:root:tot event weight 0.7425658463084365 

INFO:root:Finding ZJetsToQQ_HT-600to800 samples and should combine them under WZQQ
INFO:root:Applying lepmiso selection on 543 events
INFO:root:Applying msoftdrop selection on 543 events
INFO:root:Applying THWW selection on 478 events
INFO:root:Will fill the WZQQ dataframe with the remaining 6 events
INFO:root:tot event weight 0.5825421728617252 

INFO:root:Finding DYJetsToLL_LHEFilterPtZ-400To650 samples and should combine them under DYJets
INFO:root:Applying lepmiso selection on 24094 events
INFO:root:Applying msoftdrop selection on 24094 events
INFO:root:Applying THWW selection on 17629 events
INFO:root:Will fill the DYJets dataframe with the remaining 192 events
INFO:root:tot event weight 0.6238426296323065 

INFO:root:Finding EWKWminus_WToQQ samples and should combine them under EWKvjets
INFO:root:Applying lepmiso select

...

INFO:root:Finding GluGluHToWW_Pt-200ToInf_M-125 samples and should combine them under ggF
INFO:root:Applying msoftdrop selection on 4679 events
INFO:root:Applying THWW selection on 4005 events
INFO:root:Will fill the ggF dataframe with the remaining 1548 events
INFO:root:tot event weight 18.39408384427981 

INFO:root:Finding DYJetsToLL_LHEFilterPtZ-650ToInf samples and should combine them under DYJets
INFO:root:Applying msoftdrop selection on 220597 events
INFO:root:Applying THWW selection on 184299 events
INFO:root:Will fill the DYJets dataframe with the remaining 607 events
INFO:root:tot event weight 0.3381394460167431 

INFO:root:Finding WJetsToQQ_HT-200to400 samples and should combine them under WZQQ
INFO:root:Finding ST_tW_antitop_5f_inclusiveDecays samples and should combine them under SingleTop
INFO:root:Applying msoftdrop selection on 5306 events
INFO:root:Applying THWW selection on 4610 events
INFO:root:Will fill the SingleTop dataframe with the remaining 84 events
INFO:root:t

INFO:root:Finding DYJetsToLL_LHEFilterPtZ-400To650 samples and should combine them under DYJets
INFO:root:Applying msoftdrop selection on 198722 events
INFO:root:Applying THWW selection on 168379 events
INFO:root:Will fill the DYJets dataframe with the remaining 445 events
INFO:root:tot event weight 2.4132616341758535 

INFO:root:Finding EWKWminus_WToQQ samples and should combine them under EWKvjets
INFO:root:Applying msoftdrop selection on 68 events
INFO:root:Applying THWW selection on 55 events
INFO:root:Will fill the EWKvjets dataframe with the remaining 1 events
INFO:root:tot event weight 0.060282630879328274 

INFO:root:Finding QCD_Pt_170to300 samples and should combine them under QCD
INFO:root:Applying msoftdrop selection on 98 events
INFO:root:Applying THWW selection on 64 events
INFO:root:Will fill the QCD dataframe with the remaining 1 events
INFO:root:tot event weight 305.09025017924813 

INFO:root:Finding WJetsToLNu_HT-600To800 samples and should combine them under WJetsLNu


INFO:root:Applying lepmiso selection on 222468 events
INFO:root:Applying msoftdrop selection on 222468 events
INFO:root:Applying THWW selection on 204400 events
INFO:root:Will fill the WJetsLNu dataframe with the remaining 6189 events
INFO:root:tot event weight 421.65453878598214 

INFO:root:Finding TTToSemiLeptonic samples and should combine them under TTbar
INFO:root:Applying lepmiso selection on 402515 events
INFO:root:Applying msoftdrop selection on 402515 events
INFO:root:Applying THWW selection on 376212 events
INFO:root:Will fill the TTbar dataframe with the remaining 4244 events
INFO:root:tot event weight 667.0126272819346 

INFO:root:Finding ST_t-channel_top_4f_InclusiveDecays samples and should combine them under SingleTop
INFO:root:Applying lepmiso selection on 49730 events
INFO:root:Applying msoftdrop selection on 49730 events
INFO:root:Applying THWW selection on 47184 events
INFO:root:Will fill the SingleTop dataframe with the remaining 179 events
INFO:root:tot event weigh

INFO:root:Applying msoftdrop selection on 165755 events
INFO:root:Applying THWW selection on 122415 events
INFO:root:Will fill the DYJets dataframe with the remaining 888 events
INFO:root:tot event weight 7.510692406959251 

INFO:root:Finding ST_t-channel_antitop_4f_InclusiveDecays samples and should combine them under SingleTop
INFO:root:Applying lepmiso selection on 19318 events
INFO:root:Applying msoftdrop selection on 19318 events
INFO:root:Applying THWW selection on 18239 events
INFO:root:Will fill the SingleTop dataframe with the remaining 69 events
INFO:root:tot event weight 2.767109610883305 

INFO:root:Finding TTTo2L2Nu samples and should combine them under TTbar
INFO:root:Applying lepmiso selection on 109127 events
INFO:root:Applying msoftdrop selection on 109127 events
INFO:root:Applying THWW selection on 101815 events
INFO:root:Will fill the TTbar dataframe with the remaining 608 events
INFO:root:tot event weight 28.669486244533513 

INFO:root:Finding QCD_Pt_2400to3200 samp

INFO:root:tot event weight 41.03526980262381 

INFO:root:Finding EGamma_Run2018B samples and should combine them under Data
INFO:root:Finding DYJetsToLL_LHEFilterPtZ-0To50 samples and should combine them under DYJets
INFO:root:Applying lepmiso selection on 833 events
INFO:root:Applying msoftdrop selection on 833 events
INFO:root:Applying THWW selection on 781 events
INFO:root:Will fill the DYJets dataframe with the remaining 22 events
INFO:root:tot event weight 11.81199593527275 

INFO:root:Finding SingleMuon_Run2018C samples and should combine them under Data
INFO:root:Applying lepmiso selection on 18506 events
INFO:root:Applying msoftdrop selection on 18506 events
INFO:root:Applying THWW selection on 15328 events
INFO:root:Will fill the Data dataframe with the remaining 285 events
INFO:root:tot event weight 285.0 

INFO:root:Finding WJetsToQQ_HT-400to600 samples and should combine them under WZQQ
INFO:root:Finding SingleMuon_Run2018D samples and should combine them under Data
INFO:ro

In [90]:
def make_composition_table(events_dict, region_sel, add_soverb=False, use_Fake=False):

    from collections import OrderedDict
    
    all_MCsamples = [
        "ggF", 
        "VBF",
        "ttH",
        "WH",
        "ZH",        
        "WJetsLNu",
        "TTbar",
        "SingleTop",
        "Diboson",
        "EWKvjets",
        "DYJets",
        "WZQQ",
        "QCD",
    ]
    bkg = ["TTbar", "WJetsLNu", "Others"]
    sig = ["VBF", "ggF", "WH", "ZH", "ttH"]

    mass_window = [50, 150]
    
    # make table
    print("\\begin{table}[!ht]")
    print("\\begin{center}")
    
    
    if len(years) == 1:
        if add_soverb:
            print("\\caption{Event yield contribution from different processes in the search regions for " + years[0] + ". The last column includes $s/\sqrt{b}$ in a mass window of 100-150\GeV in the mass observable (the Higgs reconstructed mass).}")
        else:
            print("\\caption{Event yield contribution from different processes in the search regions for " +  years[0] + ".}")
    else:
        if add_soverb:
            print("\\caption{Event yield contribution from different processes in the search regions for the full Run2 dataset. The last column includes $s/\sqrt{b}$ in a mass window of 100-150\GeV in the mass observable (the Higgs reconstructed mass).}")
        else:
            print("\\caption{Event yield contribution from different processes in the search regions for the full Run2 dataset.}")
        
    if add_soverb:
        print("\\begin{tabular}{c|ccccc|ccc|c}")
    else:
        print("\\begin{tabular}{c|ccccc|ccc}")

    if add_soverb:    
        print("& \\multicolumn{5}{c|}{Signal yield} & \\multicolumn{3}{c|}{Background yield} & \\multicolumn{1}{c}{$s/\sqrt{b}$} \\\\\n")
        print("& VBF & ggF & WH & ZH & \\ttH & \\ttbar & W$+$jets & Other & \\\\\n")
    else:
        print("& \\multicolumn{5}{c|}{Signal yield} & \\multicolumn{3}{c}{Background yield} \\\\\n")
        print("& VBF & ggF & WH & ZH & \\ttH & \\ttbar & W$+$jets & Other \\\\\n")
            
    print("\\hline")
    print("\\hline")


    years_to_cover = years
    for region, sel in region_sel.items():
        if sel == 0:
            print("\\hline")
        else:
            if "2018" in region:
                years_to_cover = ["2018"]
            elif "2017" in region:
                years_to_cover = ["2017"]
            elif "2017" in region:
                years_to_cover = ["2017"]
            elif "2016APV" in region:
                years_to_cover = ["2016APV"]
            elif "2016" in region:
                years_to_cover = ["2016"]
            else:
                years_to_cover = years

            if "w/o" in region:
                ev = events_dict["without"]
            else:
                ev = events_dict["with"]

            sig_dict = OrderedDict({
                "VBF": 0,
                "ggF": 0,
                "WH": 0,
                "ZH": 0,
                "ttH": 0,            
            })

            bkg_dict = OrderedDict({
                "TTbar": 0,
                "WJetsLNu": 0, 
                "Others": 0,    
            })            

            tot_sig, tot_bkg = 0, 0
            s, b = 0, 0   # in mass window [100, 150]
            for year in years_to_cover:
                for ch in channels:
                    for sample in all_MCsamples:

                        df = ev[year][ch][sample]

                        if (ch == "mu") and (sample=="Fake"):
                            continue

                        df = df.query(sel)  

                        if sample in sig_dict:
                            tot_sig += df["nominal"].sum()                    
                            sig_dict[sample] += df["nominal"].sum()
                        elif sample in bkg_dict:
                            tot_bkg += df["nominal"].sum()
                            bkg_dict[sample] += df["nominal"].sum()
                        else:
                            tot_bkg += df["nominal"].sum()                        
                            bkg_dict["Others"] += df["nominal"].sum()                    

                        ######################## soverb start
                        df = df[(df["rec_higgs_m"]>=mass_window[0]) & (df["rec_higgs_m"]<=mass_window[1])]

                        if sample in sig_dict:                
                            s += df["nominal"].sum()
                        else:
                            b += df["nominal"].sum()
                        ######################## soverb end

            texdata = "\multirow{2}{*}{" + region + "} & \multicolumn{5}{c|}{"
            texdata += str(round(tot_sig))

            if add_soverb:
                texdata += "} & \multicolumn{3}{c|}{"
                texdata += str(round(tot_bkg))
                texdata += "} & \multicolumn{1}{c}{"            
    #             texdata += str(round(s/(b**0.5),3))
                texdata += "\multirow{2}{*}{" + str(round(s/(b**0.5),3)) + "}"

                texdata += "} \\\\\n"
            else:
                texdata += "} & \multicolumn{3}{c}{"
                texdata += str(round(tot_bkg))
                texdata += "} \\\\\n"

    #         texdata += "\\cline{2-9} \n"

            for sample in sig_dict:
    #             texdata += f" & {(100*sig_dict[sample]/tot_sig):.0f}\%"
                texdata += f" & {(sig_dict[sample]):.2f}"

            for sample in bkg_dict:
                if bkg_dict[sample]<0:
                    bkg_dict[sample]=0
    #             texdata += f" & {(100*bkg_dict[sample]/tot_bkg):.0f}\%"
                texdata += f" & {(bkg_dict[sample]):.2f}"            

            if add_soverb:
                texdata += " & "

            texdata += " \\\\\n"
            print(texdata)
            print("\\hline")

    print("\\hline")
    print("\\end{tabular}")
    print("\\label{tab:}")
    print("\\end{center}")
    print("\\end{table}")      

In [91]:
region_sel = {
    "Pre-selection (w/o jet veto maps)": f"THWW>0",
    "Pre-selection (w jet veto maps)": f"THWW>0", # dummy
    "0": 0, 

    "TopCR (w/o jet veto maps)": f"n_bjets_T>0",    
    "TopCR (w jet veto maps)": f"n_bjets_T>0",
    "1": 0,
    
    "WJetsCR (w/o jet veto maps)": f"(n_bjets_T==0) & (THWW<0.905) & (rec_higgs_pt>250)",    
    "WJetsCR (w jet veto maps)": f"(n_bjets_T==0) & (THWW<0.905) & (rec_higgs_pt>250)",
    "2": 0,
    
    "VBF cat (w/o jet veto maps)": f"(n_bjets_T==0) & (THWW>0.905) & ( (mjj>1000) & (deta>3.5) ) & (rec_higgs_pt>250)",    
    "VBF cat (w jet veto maps)": f"(n_bjets_T==0) & (THWW>0.905) & ( (mjj>1000) & (deta>3.5) ) & (rec_higgs_pt>250)",
    "3": 0,
    
    "ggF cat (w/o jet veto maps)": f"(n_bjets_T==0) & (THWW>0.905) & ( (mjj<1000) | (deta<3.5) ) & (rec_higgs_pt>250) & (VH_fj_VScore<0.9)",    
    "ggF cat (w jet veto maps)": f"(n_bjets_T==0) & (THWW>0.905) & ( (mjj<1000) | (deta<3.5) ) & (rec_higgs_pt>250) & (VH_fj_VScore<0.9)",
}

make_composition_table(events_dict, region_sel, add_soverb=True, use_Fake=False)

\begin{table}[!ht]
\begin{center}
\caption{Event yield contribution from different processes in the search regions for the full Run2 dataset. The last column includes $s/\sqrt{b}$ in a mass window of 100-150\GeV in the mass observable (the Higgs reconstructed mass).}
\begin{tabular}{c|ccccc|ccc|c}
& \multicolumn{5}{c|}{Signal yield} & \multicolumn{3}{c|}{Background yield} & \multicolumn{1}{c}{$s/\sqrt{b}$} \\

& VBF & ggF & WH & ZH & \ttH & \ttbar & W$+$jets & Other & \\

\hline
\hline
\multirow{2}{*}{Pre-selection (w/o jet veto maps)} & \multicolumn{5}{c|}{202} & \multicolumn{3}{c|}{12603} & \multicolumn{1}{c}{\multirow{2}{*}{1.861}} \\
 & 49.02 & 106.51 & 18.05 & 11.86 & 16.25 & 2933.23 & 7485.44 & 2184.48 &  \\

\hline
\multirow{2}{*}{Pre-selection (w jet veto maps)} & \multicolumn{5}{c|}{205} & \multicolumn{3}{c|}{12774} & \multicolumn{1}{c}{\multirow{2}{*}{1.879}} \\
 & 49.98 & 108.00 & 18.30 & 11.95 & 16.49 & 2985.60 & 7588.48 & 2200.07 &  \\

\hline
\hline
\multirow{2}{*}{TopCR 